## Multiple Data Imputation Exercise in R

The data in **Cancerdata** comes from a study conducted on the effect of parents' cancer on behavioral problems in their children. The dependent variable is **totbpt**, which represents the total score of behavioral problems according to Achenbach's Child Behavior Checklist (Achenbach, 1991). The variables for the cancer patient's gender (**sexP**), anxiety and depression scores of the cancer patient (**AnxtP** and **DeptP**) and the spouse (**AnxtS** and **DeptS**) are considered potential predictors of the children's behavior. Unfortunately, many of the observations are missing data (represented by the value **-9**).

(a) Considering only the complete cases, build a multiple linear regression model to relate the dependent variable (**totbpt**) to the 5 predictors mentioned above.

(b) Apply Little's test to check if the missing data follows a Completely Missing at Random (MCAR) pattern.

(c) Apply the Maximum Likelihood approach (with the EM algorithm) or the Multiple Imputation approach to estimate the missing data for all variables.

(d) Using the "complete" data, rebuild the linear regression model and compare the results with those obtained in (a).

In [25]:
library(Amelia)
library(naniar)
library(readxl) # loading libraries

Loading required package: Rcpp

## 
## Amelia II: Multiple Imputation
## (Version 1.8.3, built: 2024-11-07)
## Copyright (C) 2005-2024 James Honaker, Gary King and Matthew Blackwell
## Refer to http://gking.harvard.edu/amelia/ for more information
## 



In [4]:
path = "/home/aspphem/Desktop/MCE/StatisticalComputing/Cancer_data.xls" # file path
data <- read_excel(path) # read xlsx file

In [7]:
head(data) # data preview

SexP,DeptP,AnxtP,GSItP,DeptS,AnxtS,GSItS,SexChild,Totbpt
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,50,52,52,44,41,42,-9,-9
1,65,55,57,73,68,71,1,60
1,57,67,61,67,63,65,2,45
2,61,64,57,60,59,62,1,48
2,61,52,57,44,50,50,1,58
1,53,55,53,70,70,69,-9,-9


In [6]:
tail(data)

SexP,DeptP,AnxtP,GSItP,DeptS,AnxtS,GSItS,SexChild,Totbpt
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,63,59,61,-9,-9,-9,-9,-9
2,-9,-9,-9,-9,-9,-9,-9,-9
2,42,38,45,60,67,57,-9,-9
2,50,52,48,-9,-9,-9,-9,-9
-9,-9,-9,-9,44,55,45,-9,-9
1,73,74,72,50,46,55,-9,-9


In [11]:
dim(data) # data dimensions

[1] 89  9

In [13]:
data[data == -9] <- NA 
head(data) # replace -9 values for NAN values

SexP,DeptP,AnxtP,GSItP,DeptS,AnxtS,GSItS,SexChild,Totbpt
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,50,52,52,44,41,42,NA,NA
1,65,55,57,73,68,71,1,60
1,57,67,61,67,63,65,2,45
2,61,64,57,60,59,62,1,48
2,61,52,57,44,50,50,1,58
1,53,55,53,70,70,69,NA,NA


In [15]:
data_updated <- na.omit(data)
dim(data_updated) # drop NAN values

[1] 26  9

### Multiple Linear Regression Model

In [21]:
linear_regression <- lm(Totbpt ~ SexP + AnxtP + DeptP + AnxtS + DeptS, data = data_updated) # fit a linear regression model
summary(linear_regression) # print the result of the linear regression model


Call:
lm(formula = Totbpt ~ SexP + AnxtP + DeptP + AnxtS + DeptS, data = data_updated)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.2936  -3.2199   0.5605   4.3432   7.0916 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.93924   12.00345  -0.245 0.809055    
SexP        -3.76858    2.80346  -1.344 0.193914    
AnxtP       -0.06422    0.16908  -0.380 0.708075    
DeptP        0.88845    0.20224   4.393 0.000281 ***
AnxtS        0.60805    0.16604   3.662 0.001548 ** 
DeptS       -0.35464    0.15540  -2.282 0.033572 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 6.08 on 20 degrees of freedom
Multiple R-squared:  0.6583,	Adjusted R-squared:  0.5729 
F-statistic: 7.706 on 5 and 20 DF,  p-value: 0.0003527


The effect of the variables `DeptP`, `AnxtS`, and `DeptS` on the response variable `Totbpt` is statistically significant, with a significance level of **0.05** (p-value < 0.05).

### Little's missing completely at random (MCAR) test

*Little's (1988) test statistic is used to assess if data is missing completely at random (MCAR). The null hypothesis in this test is that the data is MCAR, and the test statistic is a chi-squared value.*

In [24]:
mcar_test(data)

statistic,df,p.value,missing.patterns
<dbl>,<dbl>,<dbl>,<int>
32.9153,30,0.3262298,8


According to the test statistic obtained, there is not enough evidence to reject the null hypothesis $H_0$. Therefore, we can say that the data follows a **MCAR** pattern.

### Missing Data Estimation

The imputation of missing data was performed using the **Amelia** library.

In [32]:
imputed_data <- amelia(data, m = 5, idvars = NULL, noms = c('SexP', 'SexChild'), seed = 123) # runs the bootstrap EM algorithm on incomplete data

-- Imputation 1 --

  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36

-- Imputation 2 --

  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60
 61 62 63 64 65 66 67 68 69 70 71 72 73 74

-- Imputation 3 --

  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23

-- Imputation 4 --

  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43

-- Imputation 5 --

  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56



In [33]:
summary(imputed_data) # summary of the imputed model


Amelia output with 5 imputed datasets.
Return code:  1 
Message:  Normal EM convergence. 

Chain Lengths:
--------------
Imputation 1:  36
Imputation 2:  74
Imputation 3:  23
Imputation 4:  43
Imputation 5:  56

Rows after Listwise Deletion:  26 
Rows after Imputation:  89 
Patterns of missingness in the data:  8 

Fraction Missing for original variables: 
-----------------------------------------

         Fraction Missing
SexP           0.07865169
DeptP          0.11235955
AnxtP          0.11235955
GSItP          0.11235955
DeptS          0.32584270
AnxtS          0.32584270
GSItS          0.32584270
SexChild       0.53932584
Totbpt         0.53932584



In [34]:
head(imputed_data$imputations[[1]]) # fst imputed dataset

SexP,DeptP,AnxtP,GSItP,DeptS,AnxtS,GSItS,SexChild,Totbpt
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,50,52,52,44,41,42,1,38.69638
1,65,55,57,73,68,71,1,60.00000
1,57,67,61,67,63,65,2,45.00000
2,61,64,57,60,59,62,1,48.00000
2,61,52,57,44,50,50,1,58.00000
1,53,55,53,70,70,69,2,52.25045


In [36]:
imputed_data_v1 <- imputed_data$imputations[[1]]
head(imputed_data_v1)

SexP,DeptP,AnxtP,GSItP,DeptS,AnxtS,GSItS,SexChild,Totbpt
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,50,52,52,44,41,42,1,38.69638
1,65,55,57,73,68,71,1,60.00000
1,57,67,61,67,63,65,2,45.00000
2,61,64,57,60,59,62,1,48.00000
2,61,52,57,44,50,50,1,58.00000
1,53,55,53,70,70,69,2,52.25045


In [38]:
linear_regression_impt <- lm(Totbpt ~ SexP + AnxtP + DeptP + AnxtS + DeptS, data = imputed_data_v1) # fit a linear regression model
summary(linear_regression_impt) # print the result of the linear regression model


Call:
lm(formula = Totbpt ~ SexP + AnxtP + DeptP + AnxtS + DeptS, data = imputed_data_v1)

Residuals:
     Min       1Q   Median       3Q      Max 
-12.5883  -3.8381  -0.8054   4.1194  12.7635 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5.06382    5.41074  -0.936 0.352048    
SexP        -3.50857    1.56614  -2.240 0.027746 *  
AnxtP        0.01650    0.09092   0.182 0.856393    
DeptP        0.74731    0.09082   8.229 2.28e-12 ***
AnxtS        0.68721    0.08708   7.891 1.07e-11 ***
DeptS       -0.33565    0.09161  -3.664 0.000436 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.727 on 83 degrees of freedom
Multiple R-squared:  0.7742,	Adjusted R-squared:  0.7606 
F-statistic: 56.93 on 5 and 83 DF,  p-value: < 2.2e-16


As with the previous model, the effects of the variables `DeptS`, `AnxtS`, and `DeptS` on the response are statistically significant. However, when considering a significance level of **alpha = 0.05**, the variable `SexP` is added to the list of statistically significant variables.

***

MSc Statistical Computing by Mathematics Research Center (CIMAT Monterrey)

October 2024